In [3]:
!pip3 -q install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [5]:
!pip3 install -qU langgraph langchain langchain_openai langchain_experimental


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [7]:
!pip3 install -qU --disable-pip-version-check qdrant-client pymupdf tiktoken

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.0 which is incompatible.
opentelemetry-proto 1.24.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.0 which is incompatible.


In [15]:
import os
from dotenv import load_dotenv
load_dotenv()
HUGGINGFACE_API_TOKEN=os.environ.get("HUGGINGFACE_API_TOKEN")
TAVILY_API_KEY=os.environ.get("TAVILY_API_KEY")

In [28]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint
from langchain.document_loaders import PyMuPDFLoader
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant

In [18]:
docs = PyMuPDFLoader("https://arxiv.org/pdf/2404.19553").load()

In [25]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350,
    chunk_overlap=50
)
chunked_documents = text_splitter.split_documents(docs)


In [29]:
embedding_model = FastEmbedEmbeddings()

Fetching 5 files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 77101.18it/s]


In [30]:
qdrant_vectorstore = Qdrant.from_documents(
    chunked_documents,
    embedding_model,
    location=":memory:",
    collection_name="extending_context_window_llama_3",
)

In [31]:
qdrant_retriever = qdrant_vectorstore.as_retriever()

In [32]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [14]:
llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2",
    task="text_generation",
    max_new_token=6096,
    huggingfacehub_api_token=HUGGINGFACE_API_TOKEN
)

/home/debian/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(
WARNING! max_new_token is not default parameter.
                    max_new_token was transferred to model_kwargs.
                    Please make sure that max_new_token is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/debian/.cache/huggingface/token
Login successful


In [39]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | rag_prompt | llm 
)

In [40]:
rag_chain.invoke({"question" : "What does the 'context' in 'long context' refer to?"})

"\nANSWER:\nThe term 'context' in 'long context' refers to a coherent text, such as a book or a long paper. It is the body of information from which the model extracts details for generating question-answer pairs."

In [41]:
from typing import Any, Callable, List, Optional, TypedDict, Union

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph

In [42]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}